This Static Model takes the CMAQ data as input and generates .npy downscaled data based on the given resolution which can be edited below. This resolution MUST be a multiple of 356.

In [ ]:
import numpy as np
import math
import os.path
from os import path
import pandas as pd
from scipy.spatial.distance import cdist
from scipy.stats import invgamma

%matplotlib inline

global resolution
global w_h
global s
global date

# 356x356 km grid
resolution = 4            # default: 4 km resolution
w_h = int(356/resolution) # default: 89x89 grid
s = int(w_h**2)           # default: 7921 sites
date = '12/30/16'           # default: 1/1/16

data = list(range(s))
coords_of_sites = list(range(s))
CMAQ_offset = list(range(s))

def progressBar(current, total, barLength):
    percent = float(current) * 100 / total
    arrow   = '-' * int(percent/100 * barLength - 1) + '>'
    spaces  = ' ' * (barLength - len(arrow))

    print('Progress: [%s%s] %d %%' % (arrow, spaces, percent), end='\r')

Estimations below are derived from Berrocal et. al trace plots.
We assume the multiplicative bias of the numerical model output is constant in space.
Thus, in fitting the static downscalter, we set the local adjustment Beta_1(s) to zero.

In [ ]:
#overall additive and multiplicative bias of CMAQ
beta_0, beta_1 = 3.2, 0.5

#nugget variance for white noise process
tau_2 = 0.10

#spatial decay parameter for Gaussian process wj(s)
phi_0 = .003

#relevant elements of matrix A
A_11 = 1.1
#this will be the value we predict in our model

First, we define the Gaussian Process for the model. This can take a LONG time since the covariance matrix is s^2/s^2, so we have the ability to load a previous matrix as an .npy file and we always save our new one.

In [ ]:
def kernel(phi=-.003):
    """
    Exponential kernel with spatial decay parameter phi.
    Default resolution=4km, i.e. 89x89 grid.
    """
    
    global resolution
    global w_h
    global s
    
    # checks for saved covariance matrix
    if path.exists("covariance_" + str(resolution) + ".npy"):
        with open("covariance_" + str(resolution) + ".npy", 'rb') as f:
            c = np.load(f, allow_pickle=False)
            return c

    # generates covariance matrix
    
    print("Generating coordinates for %d sites..." % (s))
    s_coords = [[0 for i in range(w_h)] for j in range(w_h)] 
    for x in range(w_h):
        for y in range(w_h):
            s_coords[x][y] = [x, y]     
    
    print("Generating %s x %s empty matrix..." % (s, s))
    cov = list(range(s))
    for i in range(s):
        progressBar(i, s, barLength = 80)
        cov[i] = list(range(s))
        
    print("Generating %s x %s covariance matrix..." % (s, s))
    for y in range(s):
        progressBar(y, s, barLength = 80)
        for x in range(s):
            cov[x][y] = np.exp(phi * math.sqrt((s_coords[y%w_h][y//w_h][0] - s_coords[x%w_h][x//w_h][0])**2 + \
                                               (s_coords[y%w_h][y//w_h][1] - s_coords[x%w_h][x//w_h][1])**2))
    
    # saves and returns new covariance matrix
    with open("covariance_" + str(resolution) + ".npy", 'wb') as f:
        np.save(f, cov, allow_pickle=False)
        return cov

Now we can generate a s-sized sample from the multivariate normal. Similar to the covariance matrix, this can take a long time, so we include the ability to load or save.

In [ ]:
# checks for saved sample
if path.exists("sample_" + str(resolution) + ".npy"):
    with open("sample_" + str(resolution) + ".npy", 'rb') as f:
        sample = np.load(f, allow_pickle=False)

# otherwise, generates new sample
else:
    with open("sample_" + str(resolution) + ".npy", 'wb') as f:
        mu = np.zeros(s)
        cov = kernel()
        print("Generating sample...")
        sample = np.random.multivariate_normal(mu, cov, 1)
        sample = sample[0]
        np.save(f, sample, allow_pickle=False)

Now we can start to work with the CMAQ data.

In [ ]:
# import and clean dataset
CMAQ_data = pd.read_csv('California_CMAQ_PM2.5_Output_all.csv', \
                         usecols = ['Date','Latitude', 'Longitude', 'PM'])

# extracts data from chosen date
data_start = 0
data_end = 0
while CMAQ_data.Date[data_start] != date:
    data_start += 1
data_end = data_start
while CMAQ_data.Date[data_end] == date:
    data_end += 1
    
CMAQ_data = CMAQ_data.drop(list(range(data_end+1,673251)))
CMAQ_data = CMAQ_data.drop(list(range(0,data_start)))

# organizes data further
CMAQ_data = {'Lat': CMAQ_data.Latitude,
             'Long': CMAQ_data.Longitude,
             'PM': CMAQ_data.PM}

# saves PM data length (this will be important soon)
PM_data_length = len(CMAQ_data['PM'])

""" FOR TESTING
print(PM_data_length)
print(len(CMAQ_data['Long']))
print(len(CMAQ_data['Lat']))
print(CMAQ_data['Long'][j])
"""

# and even further
CMAQ_coords = list(range(PM_data_length-1))
for a in range(data_start, data_end):
    CMAQ_coords[a-data_start] = [CMAQ_data['Long'][a], CMAQ_data['Lat'][a]]

We need to bind each site to a CMAQ sampling site. First, we must find the coordinates of each site. Then we can find each site's matching CMAQ sampling site. We do this in the slightly terrible method seen below.

In [ ]:
# lat: 35.0, 38.2, diff 3.2
# long: -121.7, -117.79, diff 3.91'
for y in range(w_h):
    for x in range(w_h):

        # assign longitudes and latitudes to each site
        coords_of_sites[x + w_h*y] = [-117.79 - x * (0.011189 * resolution), 35 + y * (.008994 * resolution)]

with open("coords_" + str(resolution) + ".npy", 'wb') as f:
    np.save(f, coords_of_sites)# allow_pickle=False)


# locates nearest CMAQ sampling site, replaces coords_toCMAQ lat/long data with PM2.5 data 
# from this nearest sampling site.
for y in range(w_h):
    #progressBar(y, w_h, barLength = 80)
    for x in range(w_h):
        distances = []
        distances = cdist([coords_of_sites[x + w_h*y]], CMAQ_coords, 'euclidean')
        CMAQ_offset[x + w_h*y] = CMAQ_data['PM'][np.argmin(distances) + data_start]

Finally, we can put all the pieces together.

In [ ]:
# generates noise
noise = np.random.normal(0, 0.10, s)

# final model
for i in range(s):
    data[i] = (beta_0 + A_11 * sample[i] + beta_1 * math.sqrt(CMAQ_offset[i]) + noise[i])**2

# saves data
with open("data_" + str(resolution) + ".npy", 'wb') as f:
        np.save(f, data, allow_pickle=False)

In [ ]:
print(len(data))
print(data[0:5])
print(data[len(data)-6:len(data)-1])
print(CMAQ_data['PM'])

7921
[9.756278110267564, 9.16114609157784, 10.691859891655437, 9.4359381689295, 8.82159245501556]
[24.0187866409146, 24.097423935183123, 24.17431505685213, 20.621372842767535, 21.02625270110925]
335244     5.7133
335245     6.0214
335246     6.0344
335247     8.2972
335248     8.4503
           ...   
336161    10.2465
336162    22.0778
336163    38.2411
336164    14.0042
336165     6.2717
Name: PM, Length: 922, dtype: float64


Now we need to upload and clean the monitoring station data.

In [ ]:
# uploads monitoring station data
PM_data = pd.read_csv('California_MonitoringStations_PM2.5_SimplifiedOutput.csv', \
                         usecols = ['Date','Latitude', 'Longitude', 'PM'])

# finds data for relevant date
PM_lat = []
PM_long = []
PM_PM = []
for i in range(len(PM_data.Date)):
    if PM_data.Date[i] == date:
        PM_PM.append(PM_data.PM[i])
        PM_long.append(PM_data.Longitude[i])
        PM_lat.append(PM_data.Latitude[i])

# saves PM data length, we will need this later
PM_data_length = len(PM_lat)        
        
# creates array of latlong data, we will need this later
PM_coords = list(range(PM_data_length))
for i in range(PM_data_length):
    PM_coords[i] = [[PM_long[i], PM_lat[i]]]

Finally, we can perform

In [ ]:
PM_CMAQ_diff = []
PM_downscaler_diff = []

for i in range(1):# PM_data_length):
    
    distances = []
    
    #print(cdist(PM_coords[i], CMAQ_coords, 'euclidean'))
    
    distances = cdist(PM_coords[i], CMAQ_coords, 'euclidean')
    
    print(CMAQ_data['Lat'][np.argmin(distances) + data_start])
    print(CMAQ_data['Long'][np.argmin(distances) + data_start])
    print(CMAQ_data['PM'][np.argmin(distances) + data_start] - PM_PM[i])
    
    PM_CMAQ_diff.append(CMAQ_data['PM'][np.argmin(distances) + data_start] - PM_PM[i])
    
    
    
    distances = []
    
    #print(cdist(PM_coords[i], coords_of_sites, 'euclidean'))
    
    distances = cdist(PM_coords[i], coords_of_sites, 'euclidean')
    
    
    print(coords_of_sites[np.argmin(distances)])
    print(data[np.argmin(distances)] - PM_PM[i])
    
    PM_downscaler_diff.append(data[np.argmin(distances)] - PM_PM[i])

MSE_CMAQ = (np.square(PM_CMAQ_diff)).mean()
MSE_downscaler = (np.square(PM_downscaler_diff)).mean()

print(MSE_CMAQ)
print(MSE_downscaler)

36.78691
-119.78145
-0.23810000000000286
[-119.759264, 36.7988]
-24.434525521493658
0.05669161000000136
597.0460374605249
